In [1]:
from openpyxl import workbook
import itertools
import random
from openpyxl import load_workbook
import numpy as np
from numpy import cumsum

### Generating permutations and drawing 5 random elements

In [4]:
def get_random_schedules():
    permutations = [x for x in itertools.permutations(range(1, 10))]
    schedules_chosen = []
    
    for elem in range(5):
        num = random.randrange(1, len(permutations))
        schedules_chosen.append(permutations[num])
        
    return schedules_chosen

### Calculating and inserting values into the spreadsheet 

In [5]:
def get_op_times(sheet):
    op_times = {
        'm1': [sheet.cell(row=3,column=i).value for i in range(2,11)],
        'm2': [sheet.cell(row=4,column=i).value for i in range(2,11)]
                }
    return op_times

In [8]:
def insert_operations(sheet, schedule, row, column):
    for i in range(len(schedule)):
        sheet.cell(row, column + i).value = schedule[i]

*Functions for calculating operational time, fit function, probability and distribution function*

In [9]:
def get_operational_time(schedules, op_times, num):
    m1_sum = cumsum(np.array([op_times['m1'][x - 1] for x in schedules[num]]))
    m2_sum = [op_times['m2'][x - 1] for x in schedules[num]]

    for elem in range(1, len(m2_sum[1:]) + 1):
        m2_sum[elem] = max(m2_sum[elem] + m2_sum[elem - 1], m2_sum[elem] + m1_sum[elem])
        
    return m2_sum[-1]

In [10]:
def get_fit_function(time):
    return 100 - time

In [11]:
def get_probability(value):
    fit_func_sum = sum([sheet.cell(12 + x, 12).value for x in range(5)]) 
    
    return round((value / fit_func_sum), 5)


In [12]:
def insert_parameters(sheet):
    for elem in range(5):
        sheet.cell(12 + elem, 11).value = get_operational_time(schedules, op_times, elem)
        sheet.cell(12 + elem, 12).value = get_fit_function(sheet.cell(12 + elem, 11).value)
    for elem in range(5):
        sheet.cell(12 + elem, 13).value = get_probability(sheet.cell(12 + elem, 12).value)
    
        if elem == 0:
            sheet.cell(12, 14).value = sheet.cell(12, 13).value
        else:
            sheet.cell(12 + elem, 14).value = sheet.cell(12 + elem, 13).value + sheet.cell((12 + elem - 1), 14).value

### Drawing pairs of parents

In [13]:
def get_pairs_of_parents(sheet):
    pair = {}
    list_of_pairs = []
    list_of_distr = [sheet.cell(12 + x, 14).value for x in range(5)]
    parents = []

    for i in range(5):
        for k in range(2):
            rand = random.uniform(0, 1)
            res = next(x for x, val in enumerate(list_of_distr) if val > rand)
            
            while True:
                if res in parents:
                    rand = random.uniform(0, 1)
                    res = next(x for x, val in enumerate(list_of_distr) if val > rand)
                else:
                    break
                    
            parents.append(res)
            pair[f'P{k + 1}'] = [res + 1, schedules[res]]

        parents = []
        list_of_pairs.append(pair)
        pair = {}
    
    return list_of_pairs


### Crossing

In [15]:
def crossing(num, list_of_pairs, fixed_opertions, parent_1, parent_2):
    offspring = [" " for x in range(9)]
    parent_schedule_1 = list_of_pairs[num][parent_1][1]
    parent_schedule_2 = list_of_pairs[num][parent_2][1]
    
    # Inserting operations from the first parent
    for elem in range(len(parent_schedule_1)):
        if elem == fixed_operations[0]:
            offspring[elem] = parent_schedule_1[elem]
        elif elem == fixed_operations[1]:
            offspring[elem] = parent_schedule_1[elem]
        elif elem == fixed_operations[2]:
            offspring[elem] = parent_schedule_1[elem]

    # Inserting operations from the second parent
    for elem in range(len(offspring)):
        if offspring[elem] == ' ':
            offspring[elem] = next(x for x in parent_schedule_2 if x not in offspring)
    
    return offspring

### Mutation

In [16]:
def mutation(offspring, indexes_to_swap):
    if type(offspring) != list:
        offspring = list(offspring)
    
    offspring[indexes_to_swap[0]], offspring[indexes_to_swap[1]] = offspring[indexes_to_swap[1]], offspring[indexes_to_swap[0]]
    return offspring

### Choosing four of the best among the offspring and the best parent

In [17]:
def get_best_offspring(sheet, list_of_offspring_m):
    offspring_results = []
    results_dict = {}
    
    for num, offspring in enumerate(list_of_offspring_m):
        results_dict['name'] = f'O{num + 1}'
        results_dict['schedule'] = offspring
        results_dict['fit_function'] = sheet.cell(37 + num, 24).value
        offspring_results.append(results_dict)
        results_dict = {}
        
    return sorted(offspring_results, key=lambda d: d['fit_function'], reverse=True)[:4] 

def get_best_parent(sheet, schedules):
    parents_results = []
    results_dict = {}
    
    for num, parent in enumerate(schedules):
        results_dict['name'] = f'P{num + 1}'
        results_dict['schedule'] = parent
        results_dict['fit_function'] = sheet.cell(12 + num, 12).value
        parents_results.append(results_dict)
        results_dict = {}
    return max(parents_results, key=lambda x:x['fit_function'])

### Executing the whole algorithm

In [23]:
num_of_iterations = 3

for iteration in range(num_of_iterations):
    # Drawing parents, calculating parameters for each of them and inserting values into the spreadsheet
    schedules = get_random_schedules()

    filename = 'alg_genetyczny.xlsx'
    wb = load_workbook(filename)
    sheet = wb['Sheet1']

    op_times = get_op_times(sheet)
    
    for i, schedule in enumerate(schedules):
        insert_operations(sheet, schedule, 12 + i, 2)
        
    insert_parameters(sheet)

    list_of_pairs = get_pairs_of_parents(sheet)

    for i in range(len(list_of_pairs)):
        sheet.cell(20 + i, 1).value = list_of_pairs[i]['P1'][0]
        sheet.cell(20 + i, 2).value = list_of_pairs[i]['P2'][0]
        sheet.cell(29 + i, 1).value = list_of_pairs[i]['P1'][0]
        sheet.cell(29 + i, 2).value = list_of_pairs[i]['P2'][0]

        insert_operations(sheet, list_of_pairs[i]['P1'][1], 20 + i, 3)
        insert_operations(sheet, list_of_pairs[i]['P2'][1], 20 + i, 17)

    # Crossing  

    list_of_offspring = []
    pc = 0.6

    # Creating the first five of the offspring
    fixed_operations = sorted(random.sample(range(9), 3))


    for i in range(len(list_of_pairs)):
        if random.uniform(0, 1) > pc:
            list_of_offspring.append(crossing(i, list_of_pairs, fixed_operations, 'P1', 'P2'))
        else:
            list_of_offspring.append(list_of_pairs[i]['P1'][1])

    for r in range(len(list_of_offspring)):
        offspring = list_of_offspring[r]
        insert_operations(sheet, offspring, 29 + r, 3)
        insert_operations(sheet, offspring, 37 + r, 2)

    # Creating another five of the offspring
    fixed_operations = sorted(random.sample(range(9), 3))


    for i in range(len(list_of_pairs)):
        if random.uniform(0, 1) > pc:
            list_of_offspring.append(crossing(i, list_of_pairs, fixed_operations, 'P2', 'P1'))
        else:
            list_of_offspring.append(list_of_pairs[i]['P2'][1])

    for r in range(len(list_of_offspring[5:])):
        offspring = list_of_offspring[r + len(list_of_offspring[5:])]
        insert_operations(sheet, offspring, 29 + r, 17)
        insert_operations(sheet, offspring, 42 + r, 2)

    # Mutation

    indexes_to_swap = sorted(random.sample(range(9), 2))
    pm = 0.3
    list_of_offspring_m = []

    for elem in list_of_offspring:
        if random.uniform(0, 1) > pm:
            list_of_offspring_m.append(mutation(elem, indexes_to_swap))
        else:
            list_of_offspring_m.append(elem)

    for r in range(len(list_of_offspring_m)):
        offspring_m = list_of_offspring_m[r]
        insert_operations(sheet, offspring_m, 37 + r, 14)

        sheet.cell(37 + r, 14 + c + 1).value = get_operational_time(list_of_offspring_m, op_times, r)
        sheet.cell(37 + r, 14 + c + 2).value = get_fit_function(sheet.cell(37 + r, 14 + c + 1).value)

    # Choosing four of the best among the offspring and the best parent based on the value of the fit function

    best_offspring = get_best_offspring(sheet, list_of_offspring_m)
    best_parent = get_best_parent(sheet, schedules)

    for count, elem in enumerate(best_offspring):
        sheet.cell(7 + count + (9 * iteration), 28).value = elem['name']
        sheet.cell(7 + count + (9 * iteration), 29).value = ', '.join([str(x) for x in elem['schedule']])
        sheet.cell(7 + count + (9 * iteration), 30).value = elem['fit_function']

    sheet.cell(7 + len(best_offspring) + (9 * iteration), 28).value =  best_parent['name']
    sheet.cell(7 + len(best_offspring) + (9 * iteration), 29).value = ', '.join([str(x) for x in best_parent['schedule']])
    sheet.cell(7 + len(best_offspring) + (9 * iteration), 30).value = best_parent['fit_function']

    wb.save(filename)